# Graph NN for MMWave Sensor filtering
The idea is to train a classifier to distinguish between fake points and actual ones

In [1]:
'''
    Requirements:
     * TF:      2.7.0
     * Keras:   2.7.0
'''

'''

    DO SPLIT AS PEDRO SUGGESTED

'''

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers

import optuna
import numpy as np
from scipy import stats
import json
import os

# Where datasets are stored
run_path = "data/"
experiments_path = "experiments/"

def load_datasets(init_path):
    '''
        Return dataset as list of pointclouds
        ---
        Parameters:
        * init_path: string; path to folder holding folders of datasets
                     (expected folder structure: 'init_path/run_X/labelled_mmw_run_X.json')
    '''
    data = []
    for run in sorted(os.listdir(init_path)):
        if "run_" in run:
            data.extend(json.load(open(init_path+run+"/labelled_mmw_"+run+".json"))['labelled_mmw'])
    return data

def get_data_and_label(data, points_per_cloud=200):
    '''
        Return samples for training as np.array, divided as unlabelled data and related labels.
        ---
        Parameters:
        * data: list of point clouds. (Usually loaded with function load_datasets)
        * points_per_cloud: number of points to be found in each point cloud. (Default is 200)
    '''
    d_x, d_y = [], []
    for pc in data:
        for i in range(0, len(pc), points_per_cloud): 
            if len(pc[i:i+points_per_cloud]) == points_per_cloud:
                t_ = np.array(pc[i:i+points_per_cloud])[:, :3]
                d_x.append(t_)
                d_y.append(np.array(pc[i:i+points_per_cloud], dtype=np.float32)[:, -1])
                # d_y.append(tf.one_hot(np.array(pc[i:i+points_per_cloud], dtype=np.float32)[:, -1], 2)) # One Hotted
    d_x, d_y = np.stack(d_x), np.stack(d_y)
    return d_x, d_y

dataset = load_datasets(run_path)

'''

    DO SPLIT AS PEDRO SUGGESTED

'''

# Separate Train and Test data
d_len = int(len(dataset)*0.7)
train, test = dataset[:d_len], dataset[d_len:]
# Shuffle point cloud dataset
np.random.shuffle(train)
np.random.shuffle(test)

# Get X and Y data for training
train_x, train_y = get_data_and_label(train)
test_x, test_y = get_data_and_label(test)

2023-01-26 12:09:40.706426: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib::/home/walter/.mujoco/mujoco210/bin:/usr/lib/nvidia
2023-01-26 12:09:40.706493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Dynamic Graph CNN 
#### https://github.com/WangYueFt/dgcnn
#### https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow

In [2]:
def gnn_conv2d(inputs,
            filters,
            kernel_size,
            stride=[1, 1],
            padding='SAME',
            use_xavier=True,
            stddev=1e-3,
            activation_fn=tf.nn.elu,
            bn=False):

    x = layers.Conv2D(
        filters, 
        kernel_size, 
        strides=stride, 
        padding=padding,
        activation=activation_fn,
        kernel_initializer='glorot_uniform' if use_xavier else keras.initializers.TruncatedNormal(stddev=stddev),
        bias_initializer='zeros'
    )(inputs)

    if bn: x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    return x

def gnn_dense(inputs,
            units,
            use_xavier=True,
            stddev=1e-3,
            activation_fn=tf.nn.elu,
            bn=False):
            
    x = layers.Dense(units,
        activation=activation_fn,
        kernel_initializer='glorot_uniform' if use_xavier else keras.initializers.TruncatedNormal(stddev=stddev),
        bias_initializer='zeros'
    )(inputs)

    if bn: x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    return x

def lambda_get_adj_matr(input):
    pcT = layers.Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(input)
    pc_inn = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))( (input, pcT) )
    pc2 = layers.Lambda(lambda x: tf.reduce_sum(tf.square(x), axis=-1, keepdims=True))(input)
    pc2T = layers.Lambda(lambda x: tf.transpose(x, perm=[0, 2, 1]))(pc2)
    output = layers.Lambda(lambda x: x[0] + -2 * x[1] + x[2])( (pc2, pc_inn, pc2T) )
    # Uncomment line below to use reciprocal of adj matrix (1/distance)
    # output = layers.Lambda(lambda x: tf.math.reciprocal(x))(output)
    return output

def lambda_knn(adj, k=20):
    x = layers.Lambda(lambda x: tf.math.top_k(-x[0], x[1]))( (adj, k) )
    return x.indices

def lambda_edge_feature(inputs, nn_idxs, k=20, num_points=200, num_dims=3):

    pc_central = inputs
    batch_size = tf.shape(inputs)[0]

    idx_ = layers.Lambda(lambda x: tf.range(x[0]) * x[1])( (batch_size, num_points) )
    idx_ = layers.Lambda(lambda x: tf.reshape(x[0], (x[1], 1, 1)))( (idx_, batch_size) )
    # Adding to list of idxs of k points the points themselves
    pc_temp1 = layers.Lambda(lambda x: x[0]+x[1])( (nn_idxs, idx_) )

    # Flattening of points into a list of coordinates (x,y,z)
    pc_flat = layers.Lambda(lambda x: tf.reshape(x[0], [-1, x[1]]))( (inputs, num_dims) )

    # Collect points from computed idxs
    pc_neighbors = layers.Lambda(lambda x: tf.gather(x[0], x[1]) )( (pc_flat, pc_temp1) )

    # Reshape points into shape (batch, num_points, NEW_AXIS = 1, num_dims)
    pc_central = layers.Lambda(lambda x: tf.expand_dims(x, axis=-2))(pc_central)
    # Points are repeated k-times along new dimension ==> (batch, num_points, k, num_dims)
    pc_central = layers.Lambda(lambda x: tf.tile(x[0], [1, 1, x[1], 1]))( (pc_central, k) )

    pc_temp2 = layers.Lambda(lambda x: tf.subtract(x[0], x[1]))( (pc_neighbors, pc_central) )
    edge_feature = layers.Lambda(lambda x: tf.concat((x[0], x[1]), axis=-1))((pc_central, pc_temp2))
    return edge_feature

def gnn_tnet(inputs, num_dims, tnet_shapes, bn=False):
    batch_size = tf.shape(inputs)[0]
    for filt in tnet_shapes[0]:
        x = gnn_conv2d(inputs, filters=filt, kernel_size=[1,1], bn=bn)
    x = tf.reduce_max(x, axis=-2, keepdims=True)
    for filt in tnet_shapes[1]:
        x = gnn_conv2d(inputs, filters=filt, kernel_size=[1,1], bn=bn)
    x = layers.GlobalMaxPooling2D(keepdims=True)(x)
    x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], y[2])))( [x, batch_size, x.shape[-1]] )

    for neur in tnet_shapes[2]:
        x = gnn_dense(x, neur, bn)
    
    bias = keras.initializers.Constant(np.eye(num_dims).flatten())
    x = layers.Dense(
        num_dims * num_dims,
        kernel_initializer="zeros",
        bias_initializer=bias,
    )(x)
    feat_T = layers.Reshape((num_dims, num_dims))(x)
    return feat_T

####################################################################################################################

# test_name = "test_0"

In [3]:
# function to test custom losses
def custom_loss(pred, labels):
    # loss = tf.compat.v1.losses.softmax_cross_entropy(onehot_labels=labels, logits=pred, label_smoothing=0.2)
    # classify_loss = tf.reduce_mean(loss)
    loss = tf.reduce_mean(tf.reduce_sum(tf.math.square(tf.math.subtract(labels, pred)), axis=-1))
    return loss

# Callback to save good models. Threshold is on validation accuracy.
class ValAccThresh_CB(keras.callbacks.Callback):
    def __init__(self, thresh=0.85, experiments_path="experiments/", test_name="test"):
        self.thresh = thresh
        super(keras.callbacks.Callback, self).__init__()
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
        self.current_epoch = 0
        self.experiments_path = experiments_path
        self.test_name = test_name

    def on_epoch_end(self, epoch, logs=None):
        self.current_epoch += 1
        val_key = ""
        for k in logs.keys():
            if "val" in k and "accuracy" in k:
                val_key = k
                break
        if val_key == "": print(" Validation Accuracy key not found.")

        current = logs.get(val_key)
        # current = logs.get("val_accuracy")
        if current >= self.thresh:
            self.thresh = current
            self.model.save_weights(self.experiments_path+self.test_name+"/best_weights/cp-"+str(epoch)+".ckpt")
            print(" New good model saved.")

# Callback to save history for post-processing
# filename=experiments_path+test_name+"/history.csv"
# history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

In [5]:
def build_model(inputs, 
        num_points, num_dims, k,
        tnet_shape,
        conv_gnns,
        dense_gnn,
        classes=2):
    '''
        Returns the outputs of the model to be compiled.
        ---
        Arguments: 
        * inputs:       Expected (None, 3). instance of tf.Input.
        * num_points:   Number of points per point cloud. Default is 200
        * num_dims:     Number of dimensions per point. Default is 3 (x, y, z)
        * k:            K nearest neighbors
        * tnet_shape:   Array of three lists. (each list's length is the number of layers for that section)
                        1st is a list of filters for convolutional layers before reduce_max.
                        2nd is a list of filters for convolutional layers after reduce_max.
                        3rd is a list of neurons for dense layers after max pooling.
        * conv_gnns:    list. Each row is composed of two lists.
                        1st is a list of filters for convolutional layers before computing edge features.
                        2nd is a list of filters for convolutional layers after computing edge features.
        * dense_gnn:    list of neuorns for dense layers at the end of the network.
        * classes:      number of classes to classify.
    '''

    adj = lambda_get_adj_matr(inputs)
    nn_idxs = lambda_knn(adj, k)
    edge_feats = lambda_edge_feature(inputs, nn_idxs, k, num_points, num_dims)
    feat_T = gnn_tnet(edge_feats, num_dims, tnet_shape, bn=True)
    pc_tf = layers.Dot(axes=(-1, -2))([inputs, feat_T]) # Apply affine transformation to input features

    adj = lambda_get_adj_matr(pc_tf)
    nn_idxs = lambda_knn(adj, k)
    edge_feats = lambda_edge_feature(pc_tf, nn_idxs, k, num_points, num_dims)

    for l in conv_gnns:
        x = edge_feats
        for gc_filt in l[0]:
            x = gnn_conv2d(x, gc_filt, [1,1], bn=True)
        x = tf.reduce_max(x, axis=-2, keepdims=True)
        x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], num_points, l[0][-1])))( [x, tf.shape(inputs)[0]] )

        adj = lambda_get_adj_matr(x)
        nn_idxs = lambda_knn(adj, k)
        edge_feats = lambda_edge_feature(x, nn_idxs, k, num_points, l[0][-1])
        x = edge_feats
        for gc_filt in l[1]:
            x = gnn_conv2d(x, gc_filt, [1,1], bn=True)
        x = tf.reduce_max(x, axis=-2, keepdims=True)
        x = layers.Lambda(lambda y: tf.reshape(y[0], (y[1], num_points, l[1][-1])))( [x, tf.shape(inputs)[0]] )

    for w_ in dense_gnn:
        x = gnn_dense(x, w_, bn=True)

    outputs = layers.Dense(classes, activation="softmax")(x)
    return outputs

def objective(trial):
    test_name = "test_"+str(trial.number)
    filename=experiments_path+test_name+"/history.csv"
    history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)
    ############################ HyperParameter Setup ############################
    ######################### Check build_model for docs #########################
    k = trial.suggest_int('k', 5,75) #30
    batch_size = trial.suggest_int('batch_size', 8,128) #16
    tnet_before_max = trial.suggest_int('tnet_before_max', 1,3)
    tnet_before = []
    for i in range(tnet_before_max):
        tnet_before.append(trial.suggest_int('tnet_beforemax_layer_'+str(i), 8,128))
    tnet_after = []
    tnet_after_max = trial.suggest_int('tnet_after_max', 1,3)
    for i in range(tnet_after_max):
        tnet_after.append(trial.suggest_int('tnet_aftermax_layer_'+str(i), 8,128))
    tnet_dense = []
    tnet_dense_layers = trial.suggest_int('tnet_dense_layers', 1,3)
    for i in range(tnet_dense_layers):
        tnet_dense.append(trial.suggest_int('tnet_dense_layer_'+str(i), 16,256))
    tnet_shape = [tnet_before, tnet_after, tnet_dense]

    gc_layers = trial.suggest_int('gc_layers', 1,3) #1
    conv_gnns = []
    for _ in range(gc_layers):
        before_edge_gcl = trial.suggest_int('before_edge_gcl', 1,3) #2
        after_edge_gcl = trial.suggest_int('after_edge_gcl', 1,3) #2

        bfr_edge = []
        for i in range(before_edge_gcl):
            bfr_edge.append(trial.suggest_int('before_edge_gcl_'+str(i), 8,128))
            
        aft_edge = []
        for i in range(after_edge_gcl):
            aft_edge.append(trial.suggest_int('after_edge_gcl_'+str(i), 8,128))
            
        conv_gnns.append([bfr_edge, aft_edge])
    dense_layers = trial.suggest_int('dense_layers', 1,3)
    dense_gnn = []
    for i in range(dense_layers):
        dense_gnn.append(trial.suggest_int('dense_layer_'+str(i), 16,256))
        
    lr = trial.suggest_float('lr', 0.0001, 0.1)
    steps_per_epoch=trial.suggest_int('steps_per_epoch', 15,100)

    validation_steps=25     # Static
    num_points = 200        # Static
    num_dims = 3            # Static
    ##############################################################################

    inputs = keras.Input(shape=(None, num_dims))
    
    outputs = build_model(inputs,
                    num_points, num_dims, k,
                    tnet_shape,
                    conv_gnns,
                    dense_gnn
                )
    model = keras.Model(inputs=[inputs], outputs=outputs, name="gnn_pointnet")

    opt_pi = tf.optimizers.Adam(learning_rate =  lr )
    # opt_pi = tf.optimizers.RMSprop(learning_rate =  lr )
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=opt_pi, metrics=[keras.metrics.SparseCategoricalAccuracy()])
    # model.compile(loss=tf.nn.sparse_softmax_cross_entropy_with_logits , optimizer=opt_pi, metrics=['accuracy'])
    # model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=opt_pi, metrics=['accuracy'])
    # model.compile(loss=custom_loss, optimizer=opt_pi, metrics=['accuracy'])

    checkpoint_path = experiments_path+test_name+"/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, 
        verbose=0, 
        save_weights_only=False,
        save_freq=10*batch_size)
        
    latest = tf.train.latest_checkpoint(experiments_path+test_name+"/")
    if latest:
        model.load_weights(latest)
        latest_ep = int(latest.split('/')[-1].split('-')[-1].split('.')[0])
        print(" Model loaded correctly:", latest, " - Epoch ", latest_ep)
    else:
        print(" The model could not be loaded properly: ", latest)
        model.save_weights(checkpoint_path.format(epoch=0))
        latest_ep = 0

    # Use CPU as default due to GPU's memory issues
    with tf.device('/CPU:0'):
        history = model.fit(
            train_x, 
            train_y, 
            
            initial_epoch=latest_ep,
            batch_size=batch_size, 
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,

            validation_split=0.3,
            epochs=150 - latest_ep, # Train for 150 epochs to find the configuration that can later be trained for more epochs.
            shuffle=True,
            callbacks=[ValAccThresh_CB(thresh=0.85, experiments_path=experiments_path, test_name=test_name),
                        cp_callback,
                        history_logger],
            use_multiprocessing=False,
            workers=8,
        )
    return np.mean(history.history['val_sparse_categorical_accuracy'][-10:])

In [9]:
# Actually train
storage = optuna.storages.RDBStorage(url="sqlite:///gnn.db", engine_kwargs={"connect_args": {"timeout": 5}})
study = optuna.create_study(study_name="gnn_denoising", storage=storage, load_if_exists=True, direction="maximize")
study.optimize(objective, n_trials=1000)

[I 2022-11-15 10:37:27,190] A new study created in RDB with name: gnn_denoising


 The model could not be loaded properly:  None
Epoch 1/150
11/97 [==>...........................] - ETA: 10:20 - loss: 0.8385 - sparse_categorical_accuracy: 0.5473

## Testing Configurations Manually

In [6]:
test_name = "manual_test_01"

k=27
batch_size=66
steps_per_epoch=33
validation_steps=25
lr = 0.06283

num_points=200
num_dims=3

checkpoint_path = experiments_path+test_name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=False,
    save_freq=10*steps_per_epoch)

inputs = keras.Input(shape=(None, 3))

tnet_shape = [[32], [128], [128,128]]
conv_gnns = [[[32,64], [128]]]
dense_gnn = [256,128]

outputs = build_model(inputs,
                    num_points, num_dims, k,
                    tnet_shape,
                    conv_gnns,
                    dense_gnn
                )

model = keras.Model(inputs=[inputs], outputs=outputs, name=test_name+"net")


opt_pi = tf.optimizers.Adam(learning_rate=lr)
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=opt_pi, metrics=[keras.metrics.SparseCategoricalAccuracy()])

# Try to load the model. If it does not exist, create it.
latest = tf.train.latest_checkpoint(experiments_path+test_name+"/")
if latest:
    model.load_weights(latest)
    latest_ep = int(latest.split('/')[-1].split('-')[-1].split('.')[0])
    print(" Model loaded correctly:", latest, " - Epoch ", latest_ep)
else:
    print(" The model could not be loaded properly: ", latest)
    model.save_weights(checkpoint_path.format(epoch=0))
    latest_ep = 0

# This grants no overwriting of the history file
filename=experiments_path+test_name+"/history"+str(latest_ep)+".csv"
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

# Use CPU as default due to GPU's memory issues
with tf.device('/CPU:0'):
    history = model.fit(
        train_x, 
        train_y, 
        
        initial_epoch=latest_ep,
        batch_size=batch_size, 
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,

        validation_split=0.3,
        epochs=20000,
        shuffle=True,
        callbacks=[ValAccThresh_CB(thresh=0.85, experiments_path=experiments_path, test_name=test_name), cp_callback, history_logger],
        use_multiprocessing=False,
        workers=8,
    )

2023-01-26 12:11:47.084408: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 12:11:47.085330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib::/home/walter/.mujoco/mujoco210/bin:/usr/lib/nvidia
2023-01-26 12:11:47.085547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib::/home/walter/.mujoco/mujoco210/bin:/usr/lib/nvidia
2023-01-26 12:11:47.085721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; 

 Model loaded correctly: experiments/manual_test_0/cp-0000.ckpt  - Epoch  0
Epoch 1/20000
33/33 [==============================] - 137s 4s/step - loss: 0.9204 - sparse_categorical_accuracy: 0.6007 - val_loss: 4.8578 - val_sparse_categorical_accuracy: 0.6867
Epoch 2/20000
33/33 [==============================] - 136s 4s/step - loss: 0.7410 - sparse_categorical_accuracy: 0.5908 - val_loss: 5.3035 - val_sparse_categorical_accuracy: 0.6903
Epoch 3/20000
33/33 [==============================] - 127s 4s/step - loss: 0.6860 - sparse_categorical_accuracy: 0.6463 - val_loss: 3.7385 - val_sparse_categorical_accuracy: 0.6901
Epoch 4/20000
33/33 [==============================] - 126s 4s/step - loss: 0.6757 - sparse_categorical_accuracy: 0.6384 - val_loss: 0.9808 - val_sparse_categorical_accuracy: 0.6903
Epoch 5/20000
33/33 [==============================] - 126s 4s/step - loss: 0.7143 - sparse_categorical_accuracy: 0.6338 - val_loss: 1.2035 - val_sparse_categorical_accuracy: 0.3330
Epoch 6/20000


2023-01-26 12:32:56.230199: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: experiments/manual_test_0/cp-0010.ckpt/assets
33/33 [==============================] - 141s 4s/step - loss: 0.6947 - sparse_categorical_accuracy: 0.6488 - val_loss: 1.1085 - val_sparse_categorical_accuracy: 0.6670
Epoch 11/20000
33/33 [==============================] - 130s 4s/step - loss: 0.6707 - sparse_categorical_accuracy: 0.6380 - val_loss: 2.3658 - val_sparse_categorical_accuracy: 0.6192
Epoch 12/20000
33/33 [==============================] - 130s 4s/step - loss: 0.6755 - sparse_categorical_accuracy: 0.6472 - val_loss: 4.5955 - val_sparse_categorical_accuracy: 0.6903
Epoch 13/20000
33/33 [==============================] - 129s 4s/step - loss: 0.6793 - sparse_categorical_accuracy: 0.6395 - val_loss: 50.8451 - val_sparse_categorical_accuracy: 0.4027
Epoch 14/20000
33/33 [==============================] - 128s 4s/step - loss: 0.7187 - sparse_categorical_accuracy: 0.5986 - val_loss: 175.7962 - val_sparse_categorical_accuracy: 0.3523
Epoch 15/20000
3

KeyboardInterrupt: 